In [1]:
# import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import scipy.stats
import scipy as sp
from scipy.io import loadmat

In [2]:
def data_matlabcode():
    
    x = np.zeros(20000)
    y = np.zeros(20000)
    
    x = np.reshape(x, (1,20000))
    y = np.reshape(y, (1,20000))
    
    x[0] = 0.1
    y[0] = 0.2
    
    for i in range(1,13):
        x[:,i] = x[:,i-1] * (3.78 - 3.78*x[:,i-1])
        y[:,i] = y[:,i-1] * (3.77 - 3.77*y[:,i-1])
        
    for i in range(13, 20000):
        x[:,i] = x[:,i-1] * (3.78 - 3.78*x[:,i-1] - 0.1*y[:,i-3])
        y[:,i] = y[:,i-1] * (3.77 - 3.77*y[:,i-1] - 0.1*x[:,i-1])
        
    # generate sinusodial seasonal component
    #s = np.sin(np.linspace(0, 2 * np.pi, 24 + 1)) # why 24 + 1
    #seasonality = np.tile(s[:-1], 250) # and why then exclude the last element
    #print(len(seasonality))
    
    #x += 0.1 * seasonality
    #y += 0.1 * seasonality
    
    return x, y #, np.concatenate((x,y), axis = 0)

In [3]:
# data = np.loadtxt('MackeyGlass_t17.txt')
# matlab_data = loadmat('xy.mat')

In [4]:
# # generate data
# X_data, Y_data = data_matlabcode()

# len = 1000

# # only use last 500 data points
# X_data = X_data[:,20000-len:]
# Y_data = Y_data[:,20000-len:]

# # standardise data
# #X_std = preprocessing.scale(X_data, axis = 1)
# #Y_std = preprocessing.scale(Y_data, axis = 1)

# X_std = np.reshape(X_data, (X_data.shape[1]))
# Y_std = np.reshape(Y_data, (Y_data.shape[1]))

# i = 0

# X_std = X_std[:X_data.shape[1]-i]
# Y_std = Y_std[i:]

# print(X_std.shape)
# print(Y_std.shape)

# print(np.mean(X_std))
# print(np.mean(Y_std))

# print(np.var(X_std))
# print(np.var(X_std))

In [5]:
# # generate the ESN reservoir
# inSize = outSize = 1
# resSize = 400
# a = 0.5 # leaking rate
# np.random.seed(42)
# #rvs = sp.stats.uniform(-1,2).rvs
# Win = (np.random.rand(resSize,1+inSize) - 0.5) * 1
# W = np.random.rand(resSize,resSize) - 0.5
# #Win = sp.sparse.random(resSize,1+inSize, 0.01, dtype = float, data_rvs = rvs).toarray()
# #W = sp.sparse.random(resSize,resSize, 0.01, dtype = float, data_rvs = rvs).toarray()
# print(W.shape)
# # normalizing and setting spectral radius (correct, slow):
# print('Computing spectral radius...')
# rhoW = max(abs(np.linalg.eig(W)[0]))
# print('done.')
# W *= 1.25 / rhoW

In [6]:
# trainLen = int(len/2)
# testLen = len - trainLen - i
# nForget = 20

# # allocated memory for the design (collected states) matrix
# X = np.zeros((1+inSize+resSize,trainLen-nForget))
# # set the corresponding target matrix directly
# Yt = Y_std[None,nForget+1:trainLen+1] 
# #Yt = data[None,nForget+1:trainLen+1] 
# print(Yt.shape)

In [7]:
# # run the reservoir with the data and collect X
# x = np.zeros((resSize,1))
# for t in range(trainLen):
#     u = X_std[t]
#     #u = data[t]
#     x = (1-a)*x + a*np.tanh( np.dot( Win, np.vstack((1,u)) ) + np.dot( W, x ) )
#     if t >= nForget:
#         X[:,t-nForget] = np.vstack((1,u,x))[:,0]

In [8]:
# # train the output by ridge regression
# reg = 1e-8 # regularization coefficient
# # direct equations from texts:
# #X_T = X.T
# #Wout = np.dot( np.dot(Yt,X_T), linalg.inv( np.dot(X,X_T) + \
# #    reg*np.eye(1+inSize+resSize) ) )
# # using scipy.linalg.solve:
# Wout = np.linalg.solve( np.dot(X,X.T) + reg*np.eye(1+inSize+resSize), 
#     np.dot(X,Yt.T) ).T

In [9]:
# # run the trained ESN in a generative mode. no need to initialize here, 
# # because x is initialized with training data and we continue from there.
# Y = np.zeros((outSize,testLen))
# u = X_std[trainLen]
# #u = data[trainLen]

# for t in range(testLen-1):
#     x = (1-a)*x + a*np.tanh( np.dot( Win, np.vstack((1,u)) ) + np.dot( W, x ) )
#     y = np.dot( Wout, np.vstack((1,u,x)) )
#     Y[:,t] = y
#     # generative mode:
#     #u = y
#     ## this would be a predictive mode:
#     u = X_std[trainLen+t+1]
#     #u = data[trainLen + t + 1]

In [10]:
# # compute MSE for the first errorLen time steps
# errorLen = testLen
# print(Y_std[trainLen:trainLen+errorLen+1].shape)
# mse = sum( np.square( Y_std[trainLen:trainLen+errorLen+1] - 
#     Y[0,0:errorLen] ) ) / errorLen
# print('MSE = ' + str( mse ))

In [11]:
# print(Y.shape)
# #print(np.reshape(X_std[9750:],(1,250)).shape)
# print(scipy.stats.spearmanr(np.reshape(Y_std[trainLen:trainLen+errorLen+1],(1,trainLen-i)), Y, axis = 1))
# print(scipy.stats.spearmanr(Y_std[trainLen:trainLen+errorLen+1],Y[0,0:errorLen+1]))


In [12]:
# # plot some signals
# plt.figure(1).clear()
# plt.plot( Y_std[trainLen+1+200:trainLen+testLen+1], 'g' )
# plt.plot( Y[:,200:].T, 'b' )
# plt.title('Target and generated signals $y(n)$ starting at $n=0$')
# plt.legend(['Target signal', 'Free-running predicted signal'])

# plt.figure(2).clear()
# plt.plot( X[0:20,0:200].T )
# plt.title(r'Some reservoir activations $\mathbf{x}(n)$')

# plt.figure(3).clear()
# plt.bar( np.arange(1+inSize+resSize), Wout[0].T )
# plt.title(r'Output weights $\mathbf{W}^{out}$')

# plt.show()

In [13]:
def Causal_ESN(X,Y, n_units = 400, rho = 1.8, leak_rate= 0.5, reg = 1e-8):
    
    """
    Causal_ESN purpose is to determine wether there is a causal relationship between
    two time series X and Y. Function is based on a Resevoir Computing method called
    Echo State Networks (ESN).
    
    Inputs:
    X - (N,) shaped numpy array representing an N-point long time series X
    Y - (N,) shaped numpy array representing an N-point long time series Y
    n_units - number of neurons in the resevois, by default 400
    rho - desired spectral radius of resevoir, by default 1.8
    leak_rate - leaking rate of the neurons in the resevoir, by default 0.5
    reg - magnitude of regularization parameter for least square fitting, by
          default set to 1e-8
          
    Outputs:
    corr - the correlation between the ESN predicted output and the test data
    """
    
    # get length of time series
    N = X.shape[0]
    #print('Length of time series {}'.format(N))
    
    # generate the ESN reservoir
    
    inSize = outSize = 1
    resSize = n_units
    a = leak_rate
    # set random seed
    np.random.seed(42)
    # initialise the input layer and resevoir
    Win = (np.random.rand(resSize,1+inSize) - 0.5) * 1
    W = np.random.rand(resSize,resSize) - 0.5
    # setting the spectral radius of resevoir W
    rhoW = max(abs(np.linalg.eig(W)[0]))
    W *= rho / rhoW
    
    trainLen = N // 2
    testLen = N - trainLen
    nForget = 20
    
    #print('Length of training set {}'.format(trainLen))
    #print('Length of testing set {}'.format(testLen))

    # initialise state matrix
    X_state = np.zeros((1+inSize+resSize,trainLen-nForget))
    # initialise matrix with collected data
    Yt = Y[None,nForget+1:trainLen+1]
    
    # run data through the reservoir and save X
    x = np.zeros((resSize,1))
    for t in range(trainLen):
        u = X[t]
        x = (1-a)*x + a*np.tanh( np.dot( Win, np.vstack((1,u)) ) + np.dot( W, x ) )
        if t >= nForget:
            X_state[:,t-nForget] = np.vstack((1,u,x))[:,0]
    

    # train the output by ridge regression
    Wout = np.linalg.solve(np.dot(X_state,X_state.T) + reg*np.eye(1+inSize+resSize),np.dot(X_state,Yt.T)).T
    
    # run through test data in predictive mode
    Y_test = np.zeros((outSize,testLen))
    u = X[trainLen]
    #print(trainLen)
    #print(testLen)
    #print(X[trainLen:].shape)
    for t in range(testLen-1):
        x = (1-a)*x + a*np.tanh( np.dot( Win, np.vstack((1,u)) ) + np.dot( W, x ) )
        y = np.dot(Wout, np.vstack((1,u,x)))
        Y_test[:,t] = y
        u = X[trainLen+t+1]
    
    # compute correlation between ESN output and test data
    #print(Y_test[:,1:].shape)
    #print(Y[trainLen+1:].shape)
    corr = sp.stats.spearmanr(Y[trainLen+1:],np.reshape(Y_test[:,:testLen-1], (testLen-1)))
    
    return corr

In [14]:
# print('Mackey Glass')
# print(Causal_ESN(data[:], data[:], rho = 0.6, leak_rate = 0.6, reg = 1e-10))

In [15]:
def data_matlabcode2():
    
    x = np.zeros(6000)
    y = np.zeros(6000)
    
    #x = np.reshape(x, (1,6000))
    #y = np.reshape(y, (1,6000))
    
    x[0] = 0.1
    y[0] = 0.2
    
    for i in range(1,13):
        x[i] = x[i-1] * (3.78 - 3.78*x[i-1])
        y[i] = y[i-1] * (3.77 - 3.77*y[i-1])
        
    for i in range(13, 6000):
        x[i] = x[i-1] * (3.78 - 3.78*x[i-1])# - 0.01*y[i-3])
        y[i] = y[i-1] * (3.77 - 3.77*y[i-1] - 0.8*x[i-1])
        
    # generate sinusodial seasonal component
    #s = np.sin(np.linspace(0, 2 * np.pi, 24 + 1)) # why 24 + 1
    #seasonality = np.tile(s[:-1], 250) # and why then exclude the last element
    #print(seasonality.shape)
    
    #x += 0.01 * seasonality
    #y += 0.01 * seasonality
    
    return x, y #, np.concatenate((x,y), axis = 0)

In [16]:
# print(matlab_data['xy'].shape)
# xy_matlab = matlab_data['xy']

In [17]:
def four_species_model_shiv(seasonal=False, years=12, obs_per_year=24, seasonal_strength=0.3):
    
    """
        Creates 4 specie model defined through the equations
    .. math::
        y_{1}(t+1) &= y_{1}(t)[3.9-3.9 y_{1}(t)]
        y_{2}(t+1) &= y_{2}(t)[3.6-0.4 y_{1}(t)-3.6 y_{2}(t)]
        y_{3}(t+1) &= y_{3}(t)[3.6-0.4 y_{2}(t)-3.6 y_{3}(t)]
        y_{4}(t+1) &= y_{4}(t)[3.8-0.35 y_{3}(t)-3.8 y_{4}(t)]
    :param seasonal:  whether to add seasonality to the mock time series
    :param years: number of 'years' of data to generate
    :param obs_per_year: observations per year: use this to express periodicity if data is seasonal
    :param seasonal_strength: global strength of seasonality, driving all variables
    :return: time series of each variable y1, y2, y3, y4
    """
    if seasonal:
        # sinusoidal seasonality
        s = np.sin(np.linspace(0, 2 * np.pi, obs_per_year + 1))
        seasonality = np.tile(s[:-1], years)
    else:
        seasonality = 0

    # initialisation
    y1 = np.zeros(int(obs_per_year * years))
    y2 = np.zeros(int(obs_per_year * years))
    y3 = np.zeros(int(obs_per_year * years))
    y4 = np.zeros(int(obs_per_year * years))
    y1[0], y2[0], y3[0], y4[0] = 0.4, 0.4, 0.4, 0.4

    # generate time series
    for i in range(obs_per_year * years - 1):
        y1[i + 1] = y1[i] * (3.9 - 3.9 * y1[i])
        y2[i + 1] = y2[i] * (3.6 - 0.4 * y1[i] - 3.6 * y2[i])
        y3[i + 1] = y3[i] * (3.6 - 0.4 * y2[i] - 3.6 * y3[i])
        y4[i + 1] = y4[i] * (3.8 - 0.35 * y3[i] - 3.8 * y4[i])

    if seasonal:
        y1 = y1 + seasonal_strength * seasonality
        y2 = y2 + seasonal_strength * seasonality
        y3 = y3 + seasonal_strength * seasonality
        y4 = y4 + seasonal_strength * seasonality
    return y1, y2, y3, y4

In [18]:
def circle_map(n = 6000):
    
    t = np.zeros(n)
    K = 1
    Omega = 0.7
    t[0] = 0.5
    
    for i in range(1,n):
        t[i] = (t[i-1] + Omega - (K/(2*np.pi)*np.sin(2*np.pi*t[i-1]))) % 1
    
    return t

In [19]:
def tent_map(n = 6000):
    
    t = np.zeros(n)
    mu = 1.9
    t[0] = 0.4
    for i  in range(1,n):
        if t[i-1]<0.5:
            t[i] = mu*t[i-1]
        else:
            t[i] = mu*(1-t[i-1])
    return t

In [46]:
# # generate data
X_data, Y_data = data_matlabcode2()

y1, y2, y3, y4 = four_species_model_shiv(years = 300)


circle = circle_map()
tent = tent_map()
#plt.plot(data[9900:])
#plt.show()
len = 5000

#only use last 500 data points
X_data = X_data[6000-len:]
Y_data = Y_data[6000-len:]

#standardise data
# X_std = preprocessing.scale(X_data)
# Y_std = preprocessing.scale(Y_data)

# X_std = np.reshape(X_data, (X_data.shape[1]))
# Y_std = np.reshape(Y_data, (Y_data.shape[1]))

i = 7
print('X and Y')
print(Causal_ESN(Y_data[i:],X_data[:len - i], rho = 1.8, leak_rate = 0.5, reg = 0))
# print('Mackey Glass')
# print(Causal_ESN(data[10000-len:], data[10000-len:], rho = 0.6, leak_rate = 0.6, reg = 1e-3))
# print('xy Matlab')
# print(Causal_ESN(xy_matlab[i:,0].T,xy_matlab[:len - i,1].T, rho = 1.8, leak_rate = 0.5, reg = 0))
print('Four Species Model')
print(Causal_ESN(y3[7200 - len + i:], y1[7200-len:7200 - i], rho = 0.6, leak_rate = 0.6, reg = 1e-3))
print('Circle Map')
print(Causal_ESN(circle[6000-len:], circle[6000-len:], rho = 0.6, leak_rate = 0.5, reg = 1e-3))
print('Tent Map')
print(Causal_ESN(tent[6000-len+i:], tent[6000-len:6000-i], rho = 0.9, leak_rate = 0.6, reg = 1e-3))

X and Y
SpearmanrResult(correlation=0.0054159858486573596, pvalue=0.7868148395637983)
Four Species Model
SpearmanrResult(correlation=0.40672984396444534, pvalue=4.548658096774347e-100)
Circle Map
SpearmanrResult(correlation=0.8634138488702128, pvalue=0.0)
Tent Map
SpearmanrResult(correlation=0.6034495217291427, pvalue=1.601975273151224e-247)


In [30]:
# pred_prey = np.genfromtxt('paramecium-didinium.csv', delimiter=',')

In [58]:
# pred_prey_std = preprocessing.scale(pred_prey[1:,:], axis = 0)
# pred_prey_std.shape

(71, 2)

In [147]:
# i = 9
# Causal_ESN(pred_prey_std[i:,1],pred_prey_std[:71-i,0], rho = 2, leak_rate = 0.5, reg = 1e-10)

SpearmanrResult(correlation=0.24271412680756396, pvalue=0.19624013774941565)

In [464]:
# print(pred_prey[1:,0].shape)

(71,)
